In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from netCDF4 import Dataset

stations_and_nodes = [
    {'station': '8632200', 'node': 514417, 'name': 'Kiptopeke, VA'},
    {'station': '8638901', 'node': 520058, 'name': 'CBBT, VA'},
    {'station': '8654467', 'node': 323049, 'name': 'Hatteras, NC'},
    {'station': '8724580', 'node': 316898, 'name': 'Vaca Key, FL'},
    {'station': '8726384', 'node': 183795, 'name': 'Port Manatee, FL'},
    {'station': '8656483', 'node': 357301, 'name': 'Beaufort, NC'},
    {'station': '8658120', 'node': 19758, 'name': 'Wilmington, NC'},
    {'station': '8726520', 'node': 164415, 'name': 'Tampa Bay, FL'}
]

# Load NetCDF files
withoutoffset_path = '/work2/07174/soelem/hopper/fort.63.nc'
withoffset_path = '/work2/07174/soelem/validation/saral/fort.63.nc'

try:
    withoutoffset = Dataset(withoutoffset_path, 'r')
    withoffset = Dataset(withoffset_path, 'r')
except OSError as e:
    print(f"Error loading NetCDF files: {e}")
    exit(1)

fig, axs = plt.subplots(len(stations_and_nodes), 1, figsize=(26, 12), dpi=600, sharex=True)

for i, station_info in enumerate(stations_and_nodes):
    node_index = station_info['node'] - 1  # Adjusting node index for Python indexing
    withoutoffset_data = withoutoffset.variables['zeta'][:, node_index]
    withoffset_data = withoffset.variables['zeta'][:, node_index]

    # Find the minimum length to ensure alignment
    min_length = min(len(withoutoffset_data), len(withoffset_data))

    # Adjust the data and dates array lengths
    withoutoffset_data = withoutoffset_data[:min_length]
    withoffset_data = withoffset_data[:min_length]

    # Adjust the phase by shifting
    withoutoffset_data_shifted = np.roll(withoutoffset_data, -4)[:min_length-4]
    withoffset_data_shifted = np.roll(withoffset_data, -4)[:min_length-4]
    dates = pd.date_range(start='2023-07-16', periods=min_length-4, freq='H')

    # Calculate the residual SSH
    residual_ssh = withoffset_data_shifted - withoutoffset_data_shifted

    # NOAA tide data preparation
    url = f'https://api.tidesandcurrents.noaa.gov/api/prod/datagetter?begin_date=20230716&end_date=20231031&station={station_info["station"]}&product=hourly_height&datum=MSL&time_zone=lst&units=metric&format=csv'
    try:
        noaa_data = pd.read_csv(url, parse_dates=['Date Time'], usecols=['Date Time', ' Water Level']).dropna()
        noaa_data.set_index('Date Time', inplace=True)
        noaa_data = noaa_data.reindex(dates, method='nearest')
    except Exception as e:
        print(f"Error loading NOAA data for station {station_info['station']}: {e}")
        continue

    axs[i].plot(dates, noaa_data[' Water Level'].values[:min_length-4], label='NOAA Observation', color='black', linestyle='-', linewidth=1)
    axs[i].plot(dates, withoutoffset_data_shifted, label='Without Offset', linestyle='-', linewidth=1, color='blue')
    axs[i].plot(dates, withoffset_data_shifted, label='With Offset', linestyle='-', linewidth=1, color='yellow')
    axs[i].plot(dates, residual_ssh, label='Residual SSH', linestyle='-', linewidth=1, color='red')

    axs[i].set_title(station_info['name'])
    axs[i].xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
    axs[i].tick_params(axis='x', rotation=45)
    axs[i].grid(True)

# Adjust the last axis to create space for the legend
handles, labels = axs[0].get_legend_handles_labels()
fig.legend(handles, labels, loc='upper center', bbox_to_anchor=(0.5, -0.01), ncol=3, fontsize='50')

plt.tight_layout()
plt.subplots_adjust(bottom=0.1)
plt.show()